# 4. Efficient Similarity Search    

## CSCI E-108      

### Steve Elston

In these exercises you will gain some experience with methods for efficient similarity search. A naive similarity search requires a brute-force computation of all pairwise distances, in the chosen metrics. The brute-force approach is not scalable with computational complexity for $n$ observations of $O(n^2)$.    

For massive datasets we need much more efficient methods. These methods fall into two broad categories. First **exact similarity search** or **exact nearest neighbor search** methods try to find the $k$ observations closest to a query vector, $q$. However, even the most efficient exact similarity search methods do not scale to high dimensional massive datasets. In order to perform similarity search on massive high-dimensional datasets we must use **approximate nearest neighbor search (ANNS)** algorithms. As you will see, ANNS algorithms involve compromises between query speed, memory use and accuracy. Whereas, exact methods are expected to have a recall score of 1.0, we must expect lower recall from ANNS algorithms as the price paid for high query speed and low memory use.            

Here, we will explore several widely used possibilities:        
1. [**KD trees**](https://en.wikipedia.org/wiki/K-d_tree) are a widely used tree-based data structure used for low-dimensional data. KD-trees are constructed from recursive binary splits. For a dataset with $n$ observations and dimensionality, $d$, the computational complexity is $O\big(d\ n \log(n) \big)$. While not ideal linear scaling, $O(n)$, the KD tree is a significant improvement over brute-force methods. Generally, KD-trees are considered to be efficient for large datasets with $d \le 20$. For higher dimensional data, R-trees and ball trees can be extended to higher dimensional data. We will apply ball trees for clustering and dimensionality reduction with the spectral clustering and UMAP algorithms.     
2. [**Inverted Indexes**](https://www.pinecone.io/learn/series/faiss/vector-indexes/) are a course quantization method that divides a large search space. Inverted indexes can be search very quickly but do not achieve high recall rates. Typically, these algorithms are used in conjunction with finer search algorithms.    
3. [**Product Quantization (PQ)**](https://www.pinecone.io/learn/series/faiss/product-quantization/) employs multi-level vector quantization. The PQ algorithm is provides fast queries with reasonable levels of recall. In addition, the PQ algorithm requires minimal memory, providing up to 97% compression!
4. [**Hierarchical Navigable Small Worlds (HNSW)** graphs](https://www.pinecone.io/learn/series/faiss/hnsw/) are arguably the state of the art in ANNS. These algorithms use a hierarchy of graphs of increasing detail and accuracy. There is a trade-off between memory use query speed and recall (accuracy) for HNSW. Properly configured, HNSW can achieve high recall scores with reasonable scalability.
5. [**Composite Indices**](https://www.pinecone.io/learn/series/faiss/composite-indexes/) use pipelines of to improve end to end performance, recall, query speed and memory use. These composites start with coarse quantization with inverted indices (IVF) and work toward fine ANN search in two or more steps. The number of possible pipeline combinations is quite large. In practice, there are a smaller number of widely used composites which are empirically known to work well for many problems.      



### Executing this notebook

This notebook is intended to run in Google Colab, using a GPU. You can set select the runtime environment by clicking the `Change runtime type` under the Runtime menu tab. If you are not familiar with working in Colab you can find a [quick start guide here](https://docs.google.com/document/d/1afPjc4IaeZzIqUAX20uBEk3Dt41pAP0Ebkpd53EJTaE/edit?tab=t.0).           

Before proceeding to the exercises, execute the code in the cell below to import the required packages.  

> **Note:** There are multiple possible version conflicts which arise when creating an environment where FAISS runs. You may well see errors from the pip installer. If this occurs you need to click `Restart session` under the Runtime menu tab and execute the code in the cell below again.

In [ ]:
!pip install --upgrade numpy<3
!pip install faiss-gpu-cu12
!pip install --upgrade scikit-learn
!pip install --upgrade pandas
!pip install tensorflow-datasets

In [2]:
import os
import time
import sys
import itertools
import collections
import numpy as np
import pandas as pd
import faiss
import shutil
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.neighbors import KDTree
from sklearn.model_selection import train_test_split
import math
from google.colab import files

## Similarity Search with KD Trees     

We will now explore a basic example of using a KD tree to find nearest neighbors. We will use a small (toy) dataset of characteristics and metabolic measurements taken from 442 diabetes patients. The KD tree algorithm will be used to find the nearest neighbors of the patients, based on the Euclidean distance between the observation vectors.    

### Load the dataset

The diabetes dataset is in the [Scikit Learn Datasets package](https://scikit-learn.org/stable/api/sklearn.datasets.html).This dataset has already been cleaned, missing values dealt with, and standardized (zero mean, unite variance). Therefore, we will skip the usual exploration and preparation steps.     

Execute the code in the cell below to load the dataset and split it into two data frames, one for training and one for test.  

In [3]:
datasets.load_diabetes()['data'].shape

(442, 10)

In [4]:
test_size = 32
column_names = ['age_years',
                'sex',
                'BMI',
                'Average_BP',
                'serum_cholesterol',
                'LDL',
                'HDL',
                'total_chol_over_HDL',
                'log_serum_triglycerides',
                'Glucose_level']

diabetes_train, diabetes_test = train_test_split(datasets.load_diabetes()['data'], test_size=test_size)

diabetes_train = pd.DataFrame(diabetes_train, columns=column_names)
diabetes_test = pd.DataFrame(diabetes_test, columns=column_names)
print('Dimensions of training data frame = ' + str(diabetes_train.shape))
print('Dimensions of test data frame = ' + str(diabetes_test.shape))
print(diabetes_test)

Dimensions of training data frame = (410, 10)
Dimensions of test data frame = (32, 10)
    age_years       sex       BMI  Average_BP  serum_cholesterol       LDL  \
0   -0.020045 -0.044642  0.097264   -0.005670          -0.005697 -0.023861   
1   -0.092695 -0.044642  0.028284   -0.015999           0.036958  0.024991   
2   -0.085430  0.050680 -0.040696   -0.033213          -0.081374 -0.069580   
3   -0.060003 -0.044642  0.001339   -0.029770          -0.007073 -0.021669   
4   -0.027310 -0.044642  0.080019    0.098751          -0.002945  0.018101   
5    0.045341  0.050680  0.005650    0.056301           0.064477  0.089186   
6   -0.034575 -0.044642 -0.037463   -0.060756           0.020446  0.043466   
7    0.045341  0.050680  0.060618    0.031065           0.028702 -0.047347   
8    0.030811  0.050680 -0.008362    0.004658           0.014942  0.027496   
9    0.001751  0.050680  0.011039   -0.019442          -0.016704 -0.003819   
10   0.045341  0.050680 -0.002973    0.107944          

Notice that each of the observations is a numeric vector with 10 variables. These variables have all be standardized.    

### Constructing and querying the KD-Tree   

The code in the cell below constructs the KD-tree as a KD-tree object with the required arguments. Execute the code.

In [5]:
%time KD_tree = KDTree(diabetes_train, leaf_size=10, metric='euclidean')

CPU times: user 3.08 ms, sys: 108 µs, total: 3.18 ms
Wall time: 3.34 ms


Notice how quickly the tree was constructed. This should not be surprising given the small data set and low dimensionality of the data.   

With the tree constructed we can now query the tree with the test data. The number of near neighbors is specified as 1 so that the query returns the single nearest neighbors.    

> **Note:** In this case we are interested in similarity with each of the observations used to construct the KD-tree, so we use $k=1$ nearest neighbors. For other purposes, such as constructing a nearest neighbor graph, a larger value of $k$ is used. We will encounter a number algorithms using nearest neighbor graphs in subsequent lessons.   

In [6]:
np.random.seed(4545)
%time distances, neighbors = KD_tree.query(diabetes_test, k=1)
distance_frame = pd.DataFrame({'Nearest Neighbor':neighbors.ravel(), 'Distance':distances.ravel()})
distance_frame.sort_values('Distance')

CPU times: user 2.06 ms, sys: 31 µs, total: 2.09 ms
Wall time: 1.67 ms


,Nearest Neighbor,Distance
12,150,0.022716
2,385,0.034785
26,30,0.038451
13,247,0.039752
17,60,0.043343
19,94,0.050318
9,151,0.051865
6,345,0.052095
8,343,0.053033
14,355,0.054334


Next print some summary statistics for the 20 queries just performed on the KD-tree by executing the code in the cell below.  

In [7]:
print("Stats for the tree")
print("Number of trims = %5d \nnumber of leaves = %5d \nnumber of splits = %6d" % KD_tree.get_tree_stats())

Stats for the tree
Number of trims =   263 
number of leaves =   391 
number of splits =    622


Finally, we can filter by similarities to find the nodes of the KD-tree that are similar to the new observations in the test dataset by applying a threshold to the distance measure. For the purpose of demonstration, we select an arbitrary distance threshold to filter on. Execute the code in the cell below to apply a similarity threshold and display the results.   

In [8]:
divisor = 2.5
distance_threshold = round(np.max(distance_frame.Distance)/divisor, 4)
print('With divisor = ' + str(divisor) + ' and distance threshold = ' + str(distance_threshold))
mask = distances < distance_threshold

n_samples = len(diabetes_test)
similarity_results = pd.DataFrame({'Sample':range(n_samples), 'Neighbor':neighbors.ravel(), 'Distance':distance_frame.Distance}).loc[mask.ravel(),:]
similarity_results.sort_values('Distance', inplace=True)

print('\nThe total number of similar cases = ' + str(sum(mask)[0]))
print(similarity_results)

With divisor = 2.5 and distance threshold = 0.0456

The total number of similar cases = 5
    Sample  Neighbor  Distance
12      12       150  0.022716
2        2       385  0.034785
26      26        30  0.038451
13      13       247  0.039752
17      17        60  0.043343


> **Exercise 4-01:** Question 1. Examine the results of the similarity search using the KD-tree algorithm. Given the number of splits (nodes) and number of leaves of the KD-tree from the statistics printed above, do you think this tree is shallow and wide or narrow and deep, and why?         

> **Answer:**
> 1. The tree is wide and shallow, rather than narrow and deep. The tree has a high leaf to node ratio since it has 391 leaves and only 622 splits of nodes. This means that it spreads out horizontally with numerous leaves at relatively few levels instead of developing a deep hierarchical structure.

> If `[100, 1000, 10000, 100000, 1000000]` times as many samples had been used construct and query the KD-tree, what is the expected wall clock time required for construction and query of the KD-tree? To answer this question you need to compute the computational complexity for these operations relative to the sames used in the running example. Use the cell below to compute and display a table of the results of your calculation. Your table should have columns showing the multiplier, the expected construction time and the expected query time.      

In [9]:

n_current = diabetes_train.shape[0]
print(f"Current number of training samples: {n_current}")

multipliers = [100, 1000, 10000, 100000, 1000000]
results = []

for mult in multipliers:
    n_new = n_current * mult

    construction_ratio = (n_new * np.log(n_new)) / (n_current * np.log(n_current))
    query_ratio = np.log(n_new) / np.log(n_current)

    results.append({
        'Multiplier': mult,
        'New Sample Size': n_new,
        'Construction Time Ratio': round(construction_ratio, 2),
        'Query Time Ratio': round(query_ratio, 2)
    })

complexity_table = pd.DataFrame(results)
print("\nComputational Complexity Analysis:")
print("=" * 60)
print(complexity_table)

Current number of training samples: 410

Computational Complexity Analysis:
   Multiplier  New Sample Size  Construction Time Ratio  Query Time Ratio
0         100            41000                   176.55              1.77
1        1000           410000                  2148.20              2.15
2       10000          4100000                 25309.34              2.53
3      100000         41000000                291366.77              2.91
4     1000000        410000000               3296401.21              3.30


> 2. What does the difference in growth in construction time and query time tell you about the scalability of $n\ log(n)$ vs. $log(n)$ computational complexity?  
> 3. There is wide range of nearest nearest neighbor distances found in the similarity search. Filtering by a distance threshold has reduced the number of similar candidates. Consider a case where you need to filter 10,000,000 cases down to 100 most similar to present top a user as search results. Given the scalablity of the query do you think this query and filtering can be done in a real-time manner of less than 20 seconds and why?     

> **Answers:**      
> 2. The difference is big in diffrent magnitude. the construction time grows exponentially while query time grows very slowly .This demonstrates that n logn complexity scales much worse than logn complexity. The n logn construction becomes prohibitively expensive for large datasets, while logn queries remain practical even at massive scale.

> 3. Yes, this should be possible in real time. According to the scalability analysis, it would only take roughly 2.53 times longer to query a KD-tree with 10 million samples than it would to query our present little dataset. If we assume that the current query takes milliseconds, obtaining the top 100 closest neighbors from 10 million instances should take less than 20 seconds, even with a 2.53x increase. The log n query complexity implies that once the tree is formed, you can search through even very large datasets quickly.

## Load the Sift1M benchmark dataset

To benchmark the algorithms we are working with we will use the Sift1M, from the [Sifts Project](https://github.com/Shifts-Project/shifts). which contains 1 million real-valued vectors of length 128. The Sift1M also includes 10000 random query vectors used for performance benchmarking. Sift1M is a widely used dataset for benchmarking the performance of approximate similarity search algorithms.   

There are a number of other widely used benchmark datasets for comparing the performance of ANNS algorithms. The [Gist1B benchmark](https://github.com/DeMoriarty/TorchPQ/blob/main/benchmark/turing/gist1m/README.md) is similar to Sift1M, but uses real-valued vectors of 980. The Gift1B datasets are similar to Sift1M, but contain 1 billion real-valued vectors of length 128. A more comprehensive list of ANNS benchmarks, including image and text data, can be found in the [ANN-Benchmarks](https://github.com/erikbern/ann-benchmarks) GitHub repository. Another index to benchmark datasets can be found on the [Zilli website](https://zilliz.com/glossary/ann-benchmarks).    

Execute the code in the cell below to load both the database of observations and the test query dataset.

> You may see a Json exception which you can safely ignore.  

In [10]:
sift1m_database = tfds.load('sift1m', split='database', data_dir='./sift1m_data')
sift1m_test = tfds.load('sift1m', split='test', data_dir='./sift1m_data')

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating database examples...: 0 examples [00:00, ? examples/s]

Shuffling sift1m_data/sift1m/incomplete.ULSQ2D_1.0.0/sift1m-database.tfrecord*...:   0%|          | 0/1000000 …

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling sift1m_data/sift1m/incomplete.ULSQ2D_1.0.0/sift1m-test.tfrecord*...:   0%|          | 0/10000 [00:00…

Dataset sift1m downloaded and prepared to sift1m_data/sift1m/1.0.0. Subsequent calls will reuse this data.


The Sift1M database you have imported is stored in a TensorFlow tensor data structure. To see some details of this data structure, execute the code in the cell below.  

In [11]:
print(sift1m_database)

<_PrefetchDataset element_spec={'embedding': TensorSpec(shape=(128,), dtype=tf.float32, name=None), 'index': TensorSpec(shape=(), dtype=tf.int64, name=None), 'neighbors': {'distance': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'index': TensorSpec(shape=(None,), dtype=tf.int64, name=None)}}>


FAISS is expecting a vector real value for each observation in the form of a Numpy array. We must therefore extract these vectors from the TensorFlow data structure. The code in the cell below demonstrates how to perform this operation. Execute this code and examine the results.

In [12]:
for element in sift1m_database.take(2):
    temp = np.array(element['embedding'])
    print(temp)

D = len(temp)
print(f"Length of embedding vector: {D}")

[  0.  16.  35.   5.  32.  31.  14.  10.  11.  78.  55.  10.  45.  83.
  11.   6.  14.  57. 102.  75.  20.   8.   3.   5.  67.  17.  19.  26.
   5.   0.   1.  22.  60.  26.   7.   1.  18.  22.  84.  53.  85. 119.
 119.   4.  24.  18.   7.   7.   1.  81. 106. 102.  72.  30.   6.   0.
   9.   1.   9. 119.  72.   1.   4.  33. 119.  29.   6.   1.   0.   1.
  14.  52. 119.  30.   3.   0.   0.  55.  92. 111.   2.   5.   4.   9.
  22.  89.  96.  14.   1.   0.   1.  82.  59.  16.  20.   5.  25.  14.
  11.   4.   0.   0.   1.  26.  47.  23.   4.   0.   0.   4.  38.  83.
  30.  14.   9.   4.   9.  17.  23.  41.   0.   0.   2.   8.  19.  25.
  23.   1.]
[ 14.  35.  19.  20.   3.   1.  13.  11.  16. 119.  85.   5.   0.   5.
  24.  26.   0.  27. 119.  13.   3.   9.  19.   0.   0.  11.  73.   9.
  10.   3.   5.   0.  92.  38.  17.  39.  32.   7.  15.  47. 119. 111.
  53.  27.   8.   0.   0.  52.   5.   7.  63.  51.  84.  43.   0.   1.
  12.   8.  20.  25.  33.  30.   2.   5.  59.  23.  25. 105.  25.

With the process of extracting the observation vectors into the rows of a Numpy array worked out, we must do so for all vectors. Execute the code in the following two cells to perform this operation for both the database and the test queries.

In [13]:
n_train = tf.data.experimental.cardinality(sift1m_database).numpy()

sift1m_numpy = np.empty((n_train,D))
for i,element in enumerate(sift1m_database.take(n_train)):
   sift1m_numpy[i,:] = np.array(element['embedding'])

print(f"Dimension of training array: {sift1m_numpy.shape}")

Dimension of training array: (1000000, 128)


In [14]:
n_test = tf.data.experimental.cardinality(sift1m_test).numpy()

sift1m_numpy_test = np.empty((n_test,D))
for i,element in enumerate(sift1m_test.take(n_test)):
   sift1m_numpy_test[i,:] = np.array(element['embedding'])

print(f"Dimension of test array: {sift1m_numpy_test.shape}")

Dimension of test array: (10000, 128)


## Load Data and Apply Flat Indexes  

We will now create a simple 'flat' index using Euclidean distance or L2 norm. A flat index is an exact similarity search since the distance is computed pairwise between all vectors.  

### Construct and test flat index

As a first step to working with ANNS you will construct a flat index as a basis for comparing the performance of algorithms. Flat indices are computed by an exact distance calculation. Nearest neighbors are found using exact distances so we can use to the flat index as a basis to benchmark other algorithms.    

For this example we are using the familiar L2 or Euclidean distance metric. Keep in mind that there is no reason for this metric to be the best choice for a particular problem. FAISS supports a several widely used distance metrics. You can see the choices [here](https://github.com/facebookresearch/faiss/wiki/MetricType-and-distances).

The code in the cell below finds the single ($k=1$) nearest neighbor of first query of the test dataset by the following steps:    
1. The flat index object is instantiated. You can find some documentation on the FAISS flat index and other basic indices [here](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes).   
2. The observations from the database are used to build the index. Since a flat index uses an exhaustive calculation, there is no model to train.    
3. The search of the nearest neighbor of the query is performed.   

Execute this code.

In [15]:
k=1

l2_index = faiss.IndexFlatL2(D)
l2_index.add(sift1m_numpy)

%time
dist, I = l2_index.search(sift1m_numpy_test[0,:].reshape(1,D), k)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.29 µs


The next question is what is the nearest neighbor to the query and how far is the query vector to the nearest neighbor. We can easily extract these values from the results returned by the search method. Execute the code in the cell below and examine the results.

In [16]:
print(f"The approximate nearest neighbor is: {I[0][0]} at a Euclidean distance {dist[0][0]}")

The approximate nearest neighbor is: 932085 at a Euclidean distance 54229.0


Typically one needs to find several nearest neighbors for a query vector. Execute the code in the cell below to find the 100 nearest neighbors to the query vector and displays the first 20.

In [17]:
k=100
%time
dist, I_L2 = l2_index.search(sift1m_numpy_test[0,:].reshape(1,D), k)

NNs = pd.DataFrame({'Neighbor':I_L2[0], 'Distance':dist[0]})
print(NNs.head(20))

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 4.05 µs
    Neighbor  Distance
0     932085   54229.0
1     934876   55091.0
2     561813   59531.0
3     708177   65260.0
4     706771   65697.0
5     695756   67010.0
6     435345   68247.0
7     701258   69844.0
8     455537   71441.0
9     872728   71861.0
10     36538   73344.0
11    562594   73537.0
12    908244   73581.0
13    600499   73793.0
14    893601   74016.0
15    619660   74356.0
16    562167   75124.0
17    746931   75554.0
18    565419   75634.0
19    236647   75704.0


At large scale we need to find a number of nearest neighbors for multiple query vectors. The dataset provides 10000 sample query vectors. In the interest of limiting execution time for the examples in these exercises we will subsample these queries to just 1000. For more exact benchmarking one should use the full set of queries, but we will settle for faster execution time.   

Execute the code in the cell below to perform the 1000 flat queries for the 100 nearest neighbors.

In [18]:
num_test = len(sift1m_numpy_test)
indx = np.random.randint(0, num_test, 1000)
%time dist, I_l2_1000 = l2_index.search(sift1m_numpy_test[indx,:], k)
print(I_l2_1000.shape)

CPU times: user 12.7 s, sys: 16.7 ms, total: 12.7 s
Wall time: 6.54 s
(1000, 100)


You now have a measure of query speed for the flat index. The other dimension of performance is memory consumption. Since FAISS is coded in C++ with a Python API, we cannot directly find the size of the index objects from Python. Therefore, we must store the index as a temporary file and then find and return the file size.  

Execute the code in the cell below

> *Citation:* The function used was copied from the Pinecone blog post ['Product Quantization: Compressing high-dimensional vectors by 97%'](https://www.pinecone.io/learn/series/faiss/product-quantization/).

In [19]:
def get_memory(index, digits=2):
    # write index to file
    faiss.write_index(index, './temp.index')
    # get file size
    file_size = os.path.getsize('./temp.index')
    # delete saved index
    os.remove('./temp.index')
    return round(file_size/1000000, digits)

#get_memory(l2_index)
print(f'Index size: {get_memory(l2_index)} MB')

Index size: 512.0 MB


## Coarse Inverted Indexes  

Inverted indexes provide a method to perform rapid ANN searches on a limited range of observed vectors. We will start to explore this area using simple coarse coded inverted indices. Again we will use the familiar L2 or Euclidean norm for simplicity. You can find an overview of coarse quantization with FAISS [here](https://www.pinecone.io/learn/series/faiss/vector-indexes/).    

The coarse inverted index is built by the following steps:    
1. A quantizer is defined using the L2 norm. The quantizer defines the method for computing distances
2. The flat inverted index is defined using the L2 norm quantizer. The search space of observations is divided into [**Voronoi regions**](https://en.wikipedia.org/wiki/Voronoi_diagram). A search starts by finding the nearest cell centroid(s). The result is based on this centroid, hence the inverted and coarse (approximate) nature of the search.     
3. The coarse quantizer is trained. The centroids of the Voronoi regions are found by using the k-means algorithm. We discuss the details of this algorithm later in the course.  
4. The observation vectors are added to the index, organized by the nearest Voronoi region centroid.   

Execute the code and examine the results.

In [20]:
nlist = 128  # The number of Voronoi cells for partitioning the data
quantizer = faiss.IndexFlatL2(D)  # Define method for storing and comparing vectors
index_IVF = faiss.IndexIVFFlat(quantizer, D, nlist) # Create an inverted file index with the quantizer
index_IVF.train(sift1m_numpy)  # Train the index to cluster into Voronoi cells
index_IVF.add(sift1m_numpy) # Add the vectors into the indexer

print(f'Index size: {get_memory(index_IVF)} MB')

Index size: 520.07 MB


As a next step, test the coarse IVF index with a single $k=100$ query by executing the code in the cell below.

In [21]:
k=100
%time dist, I = index_IVF.search(sift1m_numpy_test[0,:].reshape(1,D), k)

NNs_IVF = pd.DataFrame({'Neighbor':I[0], 'Distance':dist[0]})
print(NNs_IVF.head(20))

CPU times: user 1.77 ms, sys: 43 µs, total: 1.82 ms
Wall time: 1.35 ms
    Neighbor  Distance
0     932085   54229.0
1     934876   55091.0
2     695756   67010.0
3     455537   71441.0
4     872728   71861.0
5     562594   73537.0
6     908244   73581.0
7     600499   73793.0
8     893601   74016.0
9     619660   74356.0
10    562167   75124.0
11    565419   75634.0
12    236647   75704.0
13    568573   75944.0
14    931632   76645.0
15    454263   76664.0
16      3752   77004.0
17    871066   78324.0
18    721706   78655.0
19    619829   79309.0


The next question is who good is this nearest neighbor approximation. To find out, the code in the cell below computes the [**recall**](https://developers.google.com/machine-learning/crash-course/classification/accuracy-precision-recall) at several numbers of nearest neighbors, with respect to the ground truth found by the exact flat index search. With recall defined as:

$$recall = \frac{Number\ correctly\ identified\ neighbors}{Number\ true\ nearest\ neighbors}$$    

Execute the code in the cell below and examine the results.

In [22]:
def compute_recall(NN1, NN2, at=[1, 2, 10, 20, 100]):
  out = []
  for k in at:
    recall = sum([1 for i in NN1[:k] if i in NN2[:k]])/k
    out.append(recall)
  return pd.DataFrame({'at k':at, 'Recall':out})

compute_recall(list(NNs_IVF.Neighbor), list(NNs.Neighbor))

,at k,Recall
0,1,1.00
1,2,1.00
2,10,0.50
3,20,0.65
4,100,0.42


These recall statistics might be adequate for some purposes, but can be improved.       

A simple way to increase recall is to prob more Voronoi cells. Setting the `nprob` attribute of the index defines the number of neighboring Voronoi cells to prob. Execute this code and compare the results to the previous results with $nprob = 1$, the default value.  

In [23]:
index_IVF.nprobe = 8
%time dist, I = index_IVF.search(sift1m_numpy_test[0,:].reshape(1,D), k)
NNs_IVF_10prob = pd.DataFrame({'Neighbor':I[0], 'Distance':dist[0]})
compute_recall(list(NNs_IVF_10prob.Neighbor), list(NNs.Neighbor))

CPU times: user 5.6 ms, sys: 31 µs, total: 5.63 ms
Wall time: 5.57 ms


,at k,Recall
0,1,1.00
1,2,1.00
2,10,0.90
3,20,0.95
4,100,0.90


To get a better estimate of the performance of this index it is necessary to use more queries. Here you will execute the code cell below to execute the same randomly selected 1000 queries used for the exact flat index.

In [24]:
at=[1, 2, 10, 20, 100]
np.empty(len(at))

array([4.94e-324, 9.88e-324, 4.94e-323, 9.88e-323, 4.94e-322])

Now that the 100 queries have been executed at different values of k, it is time to evaluate the results. The code in the cell below evaluates recall at the values of k using two function.   
1. A function that computes the recall for a set of queries.
2. A function that iterates over all values of $k$, using the first function to compute recall, and returns the mean over all queries.      

Execute this code, and examine the recall tables computed at different `n_prob` values.

In [25]:
def rowwise_recall(NN1, NN2):
  """Function computes recall between rows of arrays of nearest neighbors"""
  recall = 0.0
  num_neighbors = len(NN1)
  for i in range(num_neighbors):
    recall += np.sum(np.isin(NN1[i,:], NN2[i,:]))
  return recall/num_neighbors

def compute_mean_recall(NN1, NN2, at=[1, 2, 10, 20, 100]):
  """Function computes recall between rows of arrays of nearest neighbors
  at several k values"""
#  print(len(at))
#  print(np.empty(5))
  recall = np.empty(len(at))
  for i, k in enumerate(at):
    recall[i] = rowwise_recall( NN1[:,:k], NN2[:,:k])/k
  return pd.DataFrame({'at k':at, 'Mean Recall':recall})

for nprob in [1,8, 16]:
  index_IVF.nprobe = nprob
  print(f"\n\n nprob: {nprob}")

  %time dist, I_IVF_1000 = index_IVF.search(sift1m_numpy_test[indx,:], k)

  print(compute_mean_recall(I_IVF_1000, I_l2_1000))



 nprob: 1
CPU times: user 1.03 s, sys: 21 ms, total: 1.06 s
Wall time: 574 ms
   at k  Mean Recall
0     1      0.60300
1     2      0.60000
2    10      0.55820
3    20      0.54520
4   100      0.50642


 nprob: 8
CPU times: user 7.53 s, sys: 9.75 ms, total: 7.54 s
Wall time: 3.88 s
   at k  Mean Recall
0     1      0.97200
1     2      0.97600
2    10      0.96690
3    20      0.96270
4   100      0.94905


 nprob: 16
CPU times: user 14.8 s, sys: 12 ms, total: 14.8 s
Wall time: 8.75 s
   at k  Mean Recall
0     1      0.99900
1     2      0.99850
2    10      0.99550
3    20      0.99470
4   100      0.99126


> **Exercise 4-02:** Examine these results and answer the following questions:    
> 1. In a few sentences explain how these benchmark results are interpreted? Be sure to discuss the benchmark reference for the recall metrics and the meaning of the values of k and why recall is always $\lt 1$.      
> 2. Why does increasing the `n_prob` hyperparameter increase the recall?
> 3. Why does the time required for the 1000 queries increase with values of `n_prob`?    
> 4. Explain why the inverted index is much faster than the flat index.   
> 5. If you had sufficient memory, which hyperparameter of this index could you increase to improve recall any why? Hint; `n_prob` is a property of the search on the index not a property of the index itself.  

> **Answers:**
> 1.        
> 2.         
> 3.          
> 4.        
> 5.            

## Product Quantization

We have now explored two basic ANNS index methods, flat or exact search and coarse quantization. We will now explore product quantization (PQ). In summary, the PQ algorithm uses the following steps:     
1. The vectors are divided into $m$ subvectors.
2. Each of the subvectors are quantized using a similar algorithm to coarse quantization.    
3. Queries are performed on the subvectors with the total distance being the sum of the subvector distances.     

You can find an overview of PQ with FAISS [here](https://www.pinecone.io/learn/series/faiss/vector-indexes/).

We will now construct and test the PQ index by these steps.    
1. Define the number of subvectors, $m=8$, and the number quantization bits for the subvectors, $nbits = 8$.    
2. Instantiate the PQ index.      
3. Train the PQ index.    
4. Add the database vectors to the index.   

Execute the code and examine the results.  

In [26]:
m = 8
nbits = 8
index_PQ = faiss.IndexPQ( D, m, nbits) # Create an PQ inverted index with the quantizer
index_PQ.train(sift1m_numpy)  # Train the index to cluster into Voronoi cells with PQ refinement
index_PQ.add(sift1m_numpy) # Add the vectors into the indexer

print(f'Index size: {get_memory(index_PQ)} MB')

Index size: 8.13 MB


Now you will test the PQ index on a single $k=100$ query by executing the code in the cell below.   

In [27]:
%time dist, I = index_PQ.search(sift1m_numpy_test[0,:].reshape(1,D), k)

NNs_PQ = pd.DataFrame({'Neighbor':I[0], 'Distance':dist[0]})
compute_recall(list(NNs_PQ.Neighbor), list(NNs.Neighbor))

CPU times: user 16.8 ms, sys: 0 ns, total: 16.8 ms
Wall time: 9.77 ms


,at k,Recall
0,1,1.00
1,2,0.50
2,10,0.40
3,20,0.45
4,100,0.38


To get a better idea of the performance of the PQ index execute the code in the cell below to compute the average recall for 1000 random queries.  

In [28]:
%time dist, I_PQ_1000 = index_PQ.search(sift1m_numpy_test[indx,:], k)
compute_mean_recall(I_PQ_1000, I_l2_1000)

CPU times: user 9.15 s, sys: 0 ns, total: 9.15 s
Wall time: 4.71 s


,at k,Mean Recall
0,1,0.27000
1,2,0.27200
2,10,0.31940
3,20,0.34785
4,100,0.42399


The recall of the basic PQ index can be improved by increasing the number of quantization levels, by specifying a larger $nbits$. Execute the code in the cell below that uses $nbits = 10$ to specify the number of quantization levels, and compare the results to the model with $nbits=8$. This code will take some time to execute given the long training time for the model.

In [29]:
m = 8
nbits = 10
index_PQ = faiss.IndexPQ( D, m, nbits) # Create an PQ inverted index with the quantizer
index_PQ.train(sift1m_numpy)  # Train the index to cluster into Voronoi cells with PQ refinement
index_PQ.add(sift1m_numpy) # Add the vectors into the indexer

print(f'Index size: {get_memory(index_PQ)} MB')

%time dist, I_PQ_1000 = index_PQ.search(sift1m_numpy_test[indx,:], k)
compute_mean_recall(I_PQ_1000, I_l2_1000)

Index size: 10.52 MB
CPU times: user 1min 5s, sys: 35.8 ms, total: 1min 5s
Wall time: 36.7 s


,at k,Mean Recall
0,1,0.34300
1,2,0.35550
2,10,0.43510
3,20,0.46225
4,100,0.52836


Using more, smaller, subvectors is another way to increase recall at the expense of memory and query speed. To see an example, execute the code in the cell below with $m=32$.

In [30]:
m = 32
nbits = 8
index_PQ = faiss.IndexPQ( D, m, nbits)
index_PQ.train(sift1m_numpy)
index_PQ.add(sift1m_numpy)

print(f'Index size: {get_memory(index_PQ)} MB')

%time dist, I_PQ_1000 = index_PQ.search(sift1m_numpy_test[indx,:], k)
compute_mean_recall(I_PQ_1000, I_l2_1000)

Index size: 32.13 MB
CPU times: user 35.4 s, sys: 27.9 ms, total: 35.4 s
Wall time: 20.2 s


,at k,Mean Recall
0,1,0.65100
1,2,0.66700
2,10,0.71630
3,20,0.73490
4,100,0.77188


> **Exercise 4-03:** Examine the benchmark results for the PQ ANNS and answer the following question.    
> 1. Why does increasing `nbits` only marginally increase the benchmark recall of the queries at only a modest increase in memory use? In formulating your answer, consider the role of the asymmetric distance calculation.
> 2. Calculate and compare the number of subvector quantization centroids, $p$ for the two choices of the hyperparameter `n_bits` used.
> 3. Why does increasing `m` provide greater recall increase than increasing `nbits`, but at a greater increase in memory use?

> **Answers:**
> 1.       
> 2.         
> 3.      

## Hierarchical Navigable Small World Graphs

The final basic index we will test is the hierarchical navigable small world network (HNSWN)  algorithm. In summary, the HNSWN comprises a hierarchy of graphs to go from coarse to finer grain search.      

You will now build an HNSWN index by the following steps.   
1. The number of neighbors for the nodes in the navigable small world graph is defined, $m=8$.
2. The HNSWN flat index model is instantiated. This model uses exact or flat search on each level of the graph hierarchy.    
3. The HNSWN model is trained on the database.
4. The observation vectors from the database are added to the model.

Execute this code and notice the results.    

In [31]:
m = 8
index_HNSW = faiss.IndexHNSWFlat(D, m) # Create an HNSWN index with flat L2 search at each level
index_HNSW.train(sift1m_numpy)  # Train the index which constructs the HNSW graph
index_HNSW.add(sift1m_numpy) # Add the vectors into the indexer

print(f'Index size: {get_memory(index_HNSW)} MB')

Index size: 592.57 MB


As the next step, execute the code in the cell below to execute a single $k=100$ query and examine the results.

In [32]:
%time dist, I = index_HNSW.search(sift1m_numpy_test[0,:].reshape(1,D), k)

NNs_HNSW = pd.DataFrame({'Neighbor':I[0], 'Distance':dist[0]})
compute_recall(list(NNs_HNSW.Neighbor), list(NNs.Neighbor))

CPU times: user 911 µs, sys: 37 µs, total: 948 µs
Wall time: 539 µs


,at k,Recall
0,1,0.00
1,2,0.00
2,10,0.30
3,20,0.30
4,100,0.24


Now, execute the code in the cell below to perform 1000 randomly selected queries and compute and display the recall statistics.

In [33]:
%time dist, I_HNSW_1000 = index_HNSW.search(sift1m_numpy_test[indx,:], k)
compute_mean_recall(I_HNSW_1000, I_l2_1000)

CPU times: user 124 ms, sys: 0 ns, total: 124 ms
Wall time: 64.5 ms


,at k,Mean Recall
0,1,0.73700
1,2,0.71150
2,10,0.65350
3,20,0.60340
4,100,0.41014


It is possible to increase the recall of a HNSWN model by increasing the number of neighbors for the nodes in the graph, $m$. To make a comparison execute the code in the cell below with $m=16$ and compare the results to the $m=8$ case.

In [34]:
m = 16
index_HNSW = faiss.IndexHNSWFlat(D, m) # Create an HNSWN index with flat L2 search at each level
index_HNSW.train(sift1m_numpy)  # Train the index which constructs the HNSW graph
index_HNSW.add(sift1m_numpy) # Add the vectors into the indexer

print(f'Index size: {get_memory(index_HNSW)} MB')

%time dist, I_HNSW_1000 = index_HNSW.search(sift1m_numpy_test[indx,:], k)
compute_mean_recall(I_HNSW_1000, I_l2_1000)

Index size: 656.26 MB
CPU times: user 154 ms, sys: 0 ns, total: 154 ms
Wall time: 78.1 ms


,at k,Mean Recall
0,1,0.81300
1,2,0.79900
2,10,0.73920
3,20,0.68755
4,100,0.48411


> **Exercise 4-04:** Based on the results of the benchmark tests above, answer the following questions:    
> 1. In the foregoing benchmark test a HNSWN is searched with a flat index. What is the role of the flat index in this search?     
> 2. Why does an hierarchy of small world networks result in fast ANNS?
> 3. How does increasing the hyperparameter `m` change the HNSWN, and why does this improve the recall?    
> 4. Why does increasing the hyperparameter `m` increase the query time and the memory required to store the index?  

> **Answers:**
> 1.        
> 2.      
> 3.        
> 4.          

## Summary of Algorithm Performance    

> **Exercise 4-05:** Now that you have worked with some of the basic indicies used for exact and approximate large scale nearest neighbor searches, you will compare these algorithms. To start, fill in the table shown with the performance characteristics of the algorithms, memory used, total time for 1000 queries, recall@20 for the 1000 queries and recall@100 for the 1000 queries. You need only report the results to 2 significant digits. You can find a summary of using tables in Markdown [here](https://www.codecademy.com/resources/docs/markdown/tables), as well as many other sources.       

| Algorithm | Memory Used MB | Query Time S | Recall @20 | Recall @100 |
| :-------- | :---------: | :--------: | :--------: | :---------: |
| Flat      |            |       |         |           |
| IVF, nprob = 1   |  |     |        |          |
| IVF, nprob = 8   |  |       |        |         |
| IVF, nprob = 16   | |       |        |          |
| PQ, m=8, nbits=8 |     |      |         |         |
|PQ, m=8, nbits=10 |    |      |       |          |
|PQ, m=32, nbits=8 |   |      |        |          |
|HSWN, m=8         |    |      |       |          |
|HSWN, m=16         |   |     |       |          |

> Examine the table of algorithm performance you have completed and answer these questions.     
> 1. In one or a few sentences, describe the general relationship you observe between memory use, query time and recall, given these results.  
> 2. In one or a few sentences, state which index has the longest query time. What are the trade-offs of using this index and why?
> 3. Consider the memory use of these algorithms. In one or a few sentences,state which class of algorithm uses significantly less memory than the other types, and what are the performance trade-offs for this algorithm and why?   
> 4. Consider the query time of these algorithms. In one or a few sentences,state which class of algorithm has significantly lower query time than the other types, and what are the performance trade-offs for this algorithm and why?

> Answers:      
> 1.     
> 2.       
> 3.     
> 4.     

## Composite Indexes

In practice, ANN search performance is improved by using a composite of indices. This approach builds on the strength of the various algorithms to improve recall, reduce memory use and improve query speed. The flow of the pipeline follows these general steps:   
1. **Preprocessing and transformation** to (optionally) normalize and orthogonalize the vector values. Commonly used algorithms include scaling, PCA and OPQ rotations.    
2. **Coarse quantization** for  to reduce computation and memory requirements. Algorithms commonly used include IVF, [inverted multi-index (IMI)](https://sites.skoltech.ru/app/data/uploads/sites/25/2014/12/TPAMI14.pdf), and HNSWN algorithms.   
3. **Fine quantization** primarily for compression to reduce memory required and improve query speed. PQ or flat search are the most commonly used algorithms.    
4. **Reorder results** by inverse distance so nearest neighbor is first on the list.      

You can find a tutorial introduction to working with composite indexes in FAISS [here](https://www.pinecone.io/learn/series/faiss/composite-indexes/)

### Coarse quantization with IMI

The first composite index we will investigate is the "OPQ32,IMI2x8,PQ32". During construction of this index the following operations are performed:     
1. The vectors are rotated to improved quantization, the "OPQ32" step. Confusingly, the notation "OPQ32" only specifies a rotation for the subsequent "PQ32" step.
2. An inverted multindex on a $2 \times 8$ grid is applied as a coarse quantizer.
3. A PQ index with $2^{32}$ quantization centroids is used as the fine quantizer.   

In [35]:
index_IVFOPQ = faiss.index_factory(D, "OPQ32,IMI2x8,PQ32") # Create the composite index
index_IVFOPQ.train(sift1m_numpy)  # Train the index
index_IVFOPQ.add(sift1m_numpy) # Add the vectors into the indexer

print(f'Index size: {get_memory(index_IVFOPQ)} MB')

Index size: 40.69 MB


In [36]:
k=100
%time dist, I = index_IVFOPQ.search(sift1m_numpy_test[0,:].reshape(1,D), k)

NNs_IVFOPQ = pd.DataFrame({'Neighbor':I[0], 'Distance':dist[0]})
compute_recall(list(NNs_IVFOPQ.Neighbor), list(NNs.Neighbor))

CPU times: user 1.34 ms, sys: 0 ns, total: 1.34 ms
Wall time: 4.72 ms


,at k,Recall
0,1,0.00
1,2,0.00
2,10,0.00
3,20,0.00
4,100,0.05


These results are far from encouraging! Fortunately, there is an easy fix. The IMI is a search on a grid with a great many cells. significantly increasing 'n_prob' will force search on a larger part of the grid at the expense of increased query time. Execute the code in the cell below that performs the search with `n_prob` set to 1024.      

In [37]:
imi = faiss.extract_index_ivf(index_IVFOPQ)  # we increase nprobe
imi.nprobe = 1024

%time dist, I = index_IVFOPQ.search(sift1m_numpy_test[0,:].reshape(1,D), k)

NNs_IVFOPQ = pd.DataFrame({'Neighbor':I[0], 'Distance':dist[0]})
compute_recall(list(NNs_IVFOPQ.Neighbor), list(NNs.Neighbor))

CPU times: user 11.6 ms, sys: 13 µs, total: 11.6 ms
Wall time: 12 ms


,at k,Recall
0,1,0.00
1,2,1.00
2,10,0.80
3,20,0.75
4,100,0.68


These results are much better! One should not be too concerned about recall@1, since this just means the algorithm did not find the single most similar vector to the query.     

With a better value of `n_prob` set, execute the code in the cell below to perform the 1000 random queries and compute and display the performance metrics.  

In [38]:
%time dist, I_IVFOPQ_1000 = index_IVFOPQ.search(sift1m_numpy_test[indx,:], k)
compute_mean_recall(I_IVFOPQ_1000, I_l2_1000)

CPU times: user 9.62 s, sys: 12.7 ms, total: 9.63 s
Wall time: 4.95 s


,at k,Mean Recall
0,1,0.69200
1,2,0.70900
2,10,0.76450
3,20,0.77710
4,100,0.80979


### Coarse quantization with IVF-HSWN

Now, we will investigate another composite index using a different coarse quantizer, the IVF4096_HNSW algorithm. During construction of this index the following operations are performed:     
1. The vectors are rotated to improved quantization, the "OPQ32" step.
2. An inverted 4096 cell index is constructed to search on the levels of a HNSW graph as a coarse quantizer.
3. A PQ index with $2^{32}$ quantization centroids is used as the fine quantizer.

Execute the code in the cell below to build this composite index.   

In [39]:
index_IVFHNSWPQ = faiss.index_factory(D, "OPQ32,IVF4096_HNSW,PQ32") # Create the composite index
index_IVFHNSWPQ.train(sift1m_numpy)  # Train the index
index_IVFHNSWPQ.add(sift1m_numpy) # Add the vectors into the indexer

print(f'Index size: {get_memory(index_IVFHNSWPQ)} MB')

Index size: 43.44 MB


Now, execute the code in the cell below to test the index on a single $k=100$ query.  

In [40]:
k=100
%time dist, I = index_IVFHNSWPQ.search(sift1m_numpy_test[0,:].reshape(1,D), k)

NNs_IVFHNSWPQ = pd.DataFrame({'Neighbor':I[0], 'Distance':dist[0]})
compute_recall(list(NNs_IVFHNSWPQ.Neighbor), list(NNs.Neighbor))

CPU times: user 792 µs, sys: 10 µs, total: 802 µs
Wall time: 569 µs


,at k,Recall
0,1,0.00
1,2,0.50
2,10,0.20
3,20,0.20
4,100,0.25


These initial results are not very good. But, we can easily improve on them at the expense of query time.     

Since we are using an inverted index, we can increase the number of Voronoi cells probed by the inverted index at the expense of increased query time. Execute the code in the cell below that increases `n_prob` to 128 and performs a single $k=100$ query on the index.

In [41]:
index_ = faiss.extract_index_ivf(index_IVFHNSWPQ)  # we increase nprobe
index_.nprobe = 128

%time dist, I = index_IVFHNSWPQ.search(sift1m_numpy_test[0,:].reshape(1,D), k)

NNs_IVFHNSWPQ = pd.DataFrame({'Neighbor':I[0], 'Distance':dist[0]})
compute_recall(list(NNs_IVFHNSWPQ.Neighbor), list(NNs.Neighbor))

CPU times: user 4.15 ms, sys: 0 ns, total: 4.15 ms
Wall time: 3.6 ms


,at k,Recall
0,1,1.00
1,2,1.00
2,10,0.80
3,20,0.75
4,100,0.77


Increasing `n_prob` has improved recall considerably.

Execute the code in the cell below to perform queries using the 1000 randomly sampled vectors.   

In [42]:
%time dist, I_IVFHNSWPQ_1000 = index_IVFHNSWPQ.search(sift1m_numpy_test[indx,:], k)
compute_mean_recall(I_IVFHNSWPQ_1000, I_l2_1000)

CPU times: user 3.1 s, sys: 5.06 ms, total: 3.1 s
Wall time: 1.58 s


,at k,Mean Recall
0,1,0.69800
1,2,0.72200
2,10,0.77310
3,20,0.78630
4,100,0.81693


We have now explored just two of many possible composite index constructions. Within the limits of memory and query time, we can create indexes with better recall performance if the solution requires it.

> **Exercise 4-06:** Now you will analyze the performance of the composite indices and compare these algorithms to the basic algorithms. To start, fill in the table shown with the performance characteristics of the composite index algorithms, memory used, total time for 1000 queries, recall@20 for the 1000 queries and recall@100 for the 1000 queries. You need only report the results to 2 significant digits.

| Index combination | Memory Used MB | Query Time S | Recall @20 | Recall @100 |
| :-------- | :---------: | :--------: | :--------: | :---------: |
| OPQ32,IMI2x8,PQ32     |        |     |       |         |
| OPQ32,IVF4096_HNSW,PQ32    |          |     |        |    |

> Compare the results for the two composite algorithms in the above table and the results for the basic indices in the table you prepared earlier and answer these questions:     
> 1. In one of a few sentences, describe how the performance characteristics of the composite indices compare to the basic indices.    
> 2. In one or a few sentences, describe the similarities and differences between the two composite indices.
> 3. Describe how the coarse quantization step of the composite indices improves the performance characteristics. How does the different choices of coarse quantizer explain the difference performance characteristics of the two composite indices.
> 4. For the IMI coarse index what hypterparameter can you change to improve overall performance and at what cost in performance? Don't confuse this question with the possibility of increasing query `n_prob` at the expense time.  
> 5. For the IVF_HNSW coarse index what two hypterparameters can you change to improve overall performance and at what cost in performance? Don't confuse this question with the possibility of increasing query `n_prob` at the expense time.
> 6. Which two performance characteristics of the composite indices does the OPQ algorithm improve and why?

> **Answers:**      
> 1.        
> 2.       
> 3.       
> 4.       
> 5.        
> 6.         

#### Copyright 2024, 2025, Stephen F Elston. All rights reserved.